In [147]:
import math
import numpy as np
import pandas as pd
from typing import List
from collections import defaultdict

Let $S(i, j)$ and $S(i+1, j)$ denote the shfit factors of a given set of submatrices.
The Z messages associated with a submatrix are mapped to a bunch of row chunks in a stride-pattern fashion. For each message, $t \in \{0, 1, \cdots, Z-1\}$, its corresponding row chunk index is calculated by
$$page\_addr(t, i, j)=I^{new}_{col_{t}} \pmod {N_{rc}}$$

where
$$I^{new}_{col_{t}} = (I^{new}_{col_{0}}+t) \pmod{Z}$$
$$I^{new}_{col_{0}} = Z - S(i, j)$$

Furthermore, each aforementioned message $t$ is located in a certain orow chunk by a word address, that is
$$word\_addr(t, i, j) =\lfloor I^{new}_{col_t} / N_{rc} \rfloor$$

In [148]:
class msgPass_buffer_ctrl:
	I_new_col_0 = None
	I_new_col_t = None
	
	def __init__(self, msgPass_sched: str, Z: int, N_rc: int, U: int, W_s: int, bm_row_num: int, bm_col_num: int):
		self.msgPass_sched = msgPass_sched
		self.Z = Z
		self.N_rc = N_rc
		self.U = U
		self.W_s = W_s
		self.bm_row_num = bm_row_num
		self.bm_col_num = bm_col_num
		self.page_waddr_vec = np.zeros(shape=(self.bm_col_num, self.bm_row_num, self.U, self.W_s), dtype=int)

	def new_col_normalisation(self, t: int, shift_factor: int):
		self.I_new_col_0 = self.Z-shift_factor
		self.I_new_col_t = (self.I_new_col_0+t) % self.Z
		
	def page_addr_gen(self, t: int, shift_factor: int):
		self.new_col_normalisation(t=t, shift_factor=shift_factor)
		if(self.msgPass_sched=="stride_sched"):
			page_addr=self.I_new_col_t % self.N_rc
		return page_addr
	
	def word_addr_gen(self, t: int, shift_factor: int):
		self.new_col_normalisation(t=t, shift_factor=shift_factor)
		if(self.msgPass_sched=="stride_sched"):
			word_addr=math.floor(self.I_new_col_t / self.N_rc)
		return word_addr
	
	

Configuration of the decoder architecture
- Parallelism in rows and columns of the target parity-check matrix (expanded from a given base matrix)
- To instantiate a container to emulate the message-pass buffer

In [149]:
baseMatrix_row_num = 3
baseMatrix_col_num = 10
Z=16
W_s=4
P_r=2
P_c=1
S_i_j = 0
S_i_plus1_j = 7
U = math.ceil(Z / (P_r*W_s)) # number of stride units
N_rc = math.ceil(Z / P_r) # number of row chunks in a submatrix, each of which contains N_{fg} stride fingers
msgPass_buffer_norm = np.zeros(shape=(N_rc*2+1, P_r), dtype=np.int32) # Depth: Region 0) N_rc num. of compV2C row chunks
																      #        Region 1) a blank row chunk as separator
																      #        Region 2) N_rc num. of permV2C row chunks

# A virtual message-pass buffer controller to generate the page and word addresses for writing back the cyclic shifted messages
msgPass_buffer_ctrl_inst = msgPass_buffer_ctrl(
    msgPass_sched="stride_sched",
    Z=Z,
    N_rc=N_rc,
    U=U,
    W_s=W_s,
    bm_row_num=baseMatrix_row_num,
    bm_col_num=baseMatrix_col_num
)

In [150]:
# 1) Page address (index of stride set)
# 2) Word address (finger index in a stride set)
page_addr = [0]*Z
word_addr = [0]*Z

def msgPass_buffer_permMsg_write(
		msgPass_sched: str,
		compMsg_vec: List[int], # Set of computed messages before getting (cyclic) shifted
		Z: int,
		shift_factor: int,
		N_rc: int,
		msgPass_buffer_inst: List[ List[int] ],
		permMsg_pageAddr_base: int # Base address of permuted messages region in msgPass_buffer
) -> List[ List[int] ]:
	for t in compMsg_vec:
		page_addr[t] = msgPass_buffer_ctrl_inst.page_addr_gen(t=t, shift_factor=shift_factor)
		word_addr[t] = msgPass_buffer_ctrl_inst.word_addr_gen(t=t, shift_factor=shift_factor)
		msgPass_buffer_inst[ page_addr[t]+permMsg_pageAddr_base ][ word_addr[t] ] = t
	return msgPass_buffer_inst


Let test an example that $S(i,j)=7$ and $S(i+1,j)=7$.

In [151]:
S_i_j = 0
S_i_plus1_j = 7
initial_compMsg_vec = [t for t in range(Z)]

permV2C_base_addr_vec = [0]*2
permV2C_base_addr_vec[0] = 0 # tentative value for ease of simulation
permV2C_base_addr_vec[1] = N_rc+1 # tentative value for ease of simulation

# For S(i, j)
msgPass_buffer_norm = msgPass_buffer_permMsg_write(
		msgPass_sched="stride_sched",
		compMsg_vec=initial_compMsg_vec,
		Z=Z,
		shift_factor=S_i_j,
		N_rc=N_rc,
		msgPass_buffer_inst=msgPass_buffer_norm,
		permMsg_pageAddr_base=permV2C_base_addr_vec[0]
)

# For S(i+1, j)
msgPass_buffer_norm = msgPass_buffer_permMsg_write(
		msgPass_sched="stride_sched",
		compMsg_vec=initial_compMsg_vec,
		Z=Z,
		shift_factor=S_i_plus1_j,
		N_rc=N_rc,
		msgPass_buffer_inst=msgPass_buffer_norm,
		permMsg_pageAddr_base=permV2C_base_addr_vec[1]
)

print(msgPass_buffer_norm)

[[ 0  8]
 [ 1  9]
 [ 2 10]
 [ 3 11]
 [ 4 12]
 [ 5 13]
 [ 6 14]
 [ 7 15]
 [ 0  0]
 [ 7 15]
 [ 8  0]
 [ 9  1]
 [10  2]
 [11  3]
 [12  4]
 [13  5]
 [14  6]]


The next section will address the cyclic shfit factor for each row chunk.

# Stride Unit Assignment

The row chunks, stride units and input sources of a page alignment unit are formulated as follows. A submatrix $b$ consists of a set of stride units, $\mathbb{S}=\{\mathbb{S}_{0}, \mathbb{S}_{1}, \cdots. \mathbb{S}_{U-1}\}$ where $U = Z / (P_{r} \cdot W^{s})$ accounts for the number of stride units. For all $u \in \{0, \cdots, U-1\}$, a set of consecutive row chunks is included in a stride unit, i.e. $\mathbb{S}_{u} = \{R^{u}_{0}, R^{u}_{2}, \cdots, R^{u}_{W^{s}-1}\}$ where $R^{u}_{0}$ denotes the first row chunk in the $u$th stride unit, etc. Moreover, a row chunk $R^{u}_{i}$ aggregates a bunch of extrinsic messages\footnote{Every extrinsic message is from a nonzero element in submatrix $b$, which represents one associated variable node.}, i.e.

$$
\forall i \in \{0, \cdots, W^{s}-1\}, \forall u \in \{0, \cdots, U-1\}, \\
R^{u}_{i} = \{y_{o} | 0 \leq j \leq P_{r}-1, j \in \mathbb{Z}^{+}, o=N_{rc}*j+i+u*W^{s}\}.
$$

# Generation of Row-Chunk Cyclic Shift Factors

This section is to determine the cyclic shift factor for the permutation of each row chunk.
Let pick up one element from each row chunk as representatives, presenting by a set $E^{pg}$,
$$
E^{pg} = \{t \in [0, Z) | t \pmod {P_{r}}=0\}.
$$

Next step is to get the page address and word address from each element of $E^{pg}$,
$$
\forall e \in E^{pg}, \\
\hat{P}^{cur}_{e} = page\_addr(e, i, j), \\
\hat{P}^{next}_{e} = page\_addr(e, i+1, j), \\
\hat{W}^{cur}_{e} = word\_addr(e, i, j), \\
\hat{W}^{next}_{e} = word\_addr(e, i+1, j). \\
$$

Finally, the cyclic shift factor for passing messages in each row chunk, from $i$-th decoding layer to $(i+1)$-th deocoding layer, can be calculated by
$$
\hat{S}(i, j, \hat{P}^{cur}_{e}) = 
\begin{cases}
    \hat{W}^{next}_{e} - \hat{W}^{cur}_{e}, & \text{if } \hat{W}^{next}_{e} \ge \hat{W}^{cur}_{e} \\
    N_{rc} + \hat{W}^{next}_{e} - \hat{W}^{cur}_{e}, & \text{otherwise}
\end{cases}
$$

Example of $submatrix(i=0, j=0)$ and $submatrix(i=1, j=0)$.

In [152]:
i=0
j=0
page_vec = [t for t in range(Z) if (t % P_r)==0]
cyclic_shiftFactor_vec= np.zeros(shape=(baseMatrix_row_num, baseMatrix_col_num, N_rc), dtype=np.int32)
for t in page_vec:
    # To generate the cyclic shift factors for row chunks
    p0 = msgPass_buffer_ctrl_inst.page_addr_gen(t=t, shift_factor=S_i_j)
    w0 = msgPass_buffer_ctrl_inst.word_addr_gen(t=t, shift_factor=S_i_j)
    p1 = msgPass_buffer_ctrl_inst.page_addr_gen(t=t, shift_factor=S_i_plus1_j)
    w1 = msgPass_buffer_ctrl_inst.word_addr_gen(t=t, shift_factor=S_i_plus1_j)
    if w1 < w0:
        hat_S_i_j_r = P_r+w1-w0
    else: # w1 >= w0
        hat_S_i_j_r = w1-w0
    print(f"hat_P_cur_e={p0}\t->\t{p0}-th row chunk gets shifted by the cyclic shift factor {hat_S_i_j_r}, p1: {p1}")
    cyclic_shiftFactor_vec[i][j][p0] = hat_S_i_j_r

print(f"cyclic_shiftFactor_vec[{i}][{j}]: {cyclic_shiftFactor_vec[i][j]}")

hat_P_cur_e=0	->	0-th row chunk gets shifted by the cyclic shift factor 1, p1: 1
hat_P_cur_e=2	->	2-th row chunk gets shifted by the cyclic shift factor 1, p1: 3
hat_P_cur_e=4	->	4-th row chunk gets shifted by the cyclic shift factor 1, p1: 5
hat_P_cur_e=6	->	6-th row chunk gets shifted by the cyclic shift factor 1, p1: 7
hat_P_cur_e=0	->	0-th row chunk gets shifted by the cyclic shift factor 1, p1: 1
hat_P_cur_e=2	->	2-th row chunk gets shifted by the cyclic shift factor 1, p1: 3
hat_P_cur_e=4	->	4-th row chunk gets shifted by the cyclic shift factor 1, p1: 5
hat_P_cur_e=6	->	6-th row chunk gets shifted by the cyclic shift factor 1, p1: 7
cyclic_shiftFactor_vec[0][0]: [1 0 1 0 1 0 1 0]


# Algorithm - Message Passing Procedures for the Computed V2C Messages

- StrideUnit-to-message converter: to obtain the indices of stride unit and relative row chunk that $t$-th message is mapped to 
$$
\forall i \in \{0, \cdots, W^{s}-1\}, \forall u \in \{0, \cdots, U-1\}, \\
R^{u}_{i} = \{y_{o} | 0 \leq j \leq P_{r}-1, j \in \mathbb{Z}^{+}, o=N_{rc}*j+i+u*W^{s}\}.
$$

- Message-to-StrideUnit converter
$$
\forall t \in [0, Z), \\

u = \lfloor (t \mod{(W^{s} \cdot U)}) / W^{s} \rfloor \\
i = (t \mod{(W^{s} \cdot U)}) \mod W^{s}
$$

---

Let assign sample values to all stride unit.

In [153]:
R = [ [[0]*P_r for i in range(W_s)] for u in range(U) ]
for u in range(U):
	for i in range(W_s):
		for j in range(P_r):
			R[u][i][j] = N_rc*j+i+u*W_s
			#print(f"R[{u}][{i}][{j}] = {R[u][i][j]}")

for t in range(Z):
	u = math.floor((t % (W_s*U)) / W_s)
	i = (t % (W_s*U)) % W_s
	#print(f"t={t} u={u} i={i}")
	if t not in R[u][i]:
		print(f"Error: t={t} u={u} i={i} not in R[{u}][{i}]")

---

To precalculate the page addresses for writing back the cyclic shifted message within $submatrix(i,j)$

In [158]:
i=0
j=0
page_vec = [t for t in range(Z) if (t % P_r)==0]
abs_rowChunk_id_vec = list()
rel_rowChunk_id_vec = list()
stride_unit_id_vec = list()
hat_abs_rowChunk_id_vec = list()
hat_rel_rowChunk_id_vec = list()
hat_stride_unit_id_vec = list()
for t in page_vec:
    abs_rowChunk_id = msgPass_buffer_ctrl_inst.page_addr_gen(t=t, shift_factor=S_i_j)
    rel_rowChunk_id = abs_rowChunk_id % W_s
    stride_unit_id = math.floor(abs_rowChunk_id / W_s)
    abs_rowChunk_id_vec.append(abs_rowChunk_id)
    rel_rowChunk_id_vec.append(rel_rowChunk_id)
    stride_unit_id_vec.append(stride_unit_id)
    #print(f"t: {t}|\t->\t| Row chunk for i-th layer: {abs_rowChunk_id} (absolute), i.e. {rel_rowChunk_id} (relative) at stride unit {stride_unit_id}")
  
    hat_abs_rowChunk_id = msgPass_buffer_ctrl_inst.page_addr_gen(t=t, shift_factor=S_i_j+S_i_plus1_j)
    hat_rel_rowChunk_id = hat_abs_rowChunk_id % W_s
    hat_stride_unit_id = math.floor(hat_abs_rowChunk_id / W_s)
    hat_abs_rowChunk_id_vec.append(hat_abs_rowChunk_id)
    hat_rel_rowChunk_id_vec.append(hat_rel_rowChunk_id)
    hat_stride_unit_id_vec.append(hat_stride_unit_id)
    #print(f"t: {t}|\t->\t| Row chunk for (i+1)-th layer: {hat_abs_rowChunk_id} (absolute), i.e. {hat_rel_rowChunk_id} (relative) at stride unit {hat_stride_unit_id}")

# To generate the page addresses for writing back the cyclic shifted messages
bm_row=i
bm_col=j

for u in stride_unit_id_vec:
    hat_u = hat_stride_unit_id_vec[u]
    for rc in rel_rowChunk_id_vec:
        hat_rc = hat_rel_rowChunk_id_vec[rc]
        msgPass_buffer_ctrl_inst.page_waddr_vec[bm_row][bm_col][u][rc] = hat_rc

print(f"msgPass_buffer_ctrl_inst.page_waddr_vec[bm_row][bm_col]: \n{msgPass_buffer_ctrl_inst.page_waddr_vec[bm_row][bm_col]}")

msgPass_buffer_ctrl_inst.page_waddr_vec[bm_row][bm_col]: 
[[1 0 1 0]
 [1 0 1 0]]


In [ ]:
msgPass_buffer = np.zeros(shape=(N_rc*2+1, P_r), dtype=np.int32) # Depth: Region 0) N_rc num. of compV2C row chunks
																 #        Region 1) a blank row chunk as separator
																 #        Region 2) N_rc num. of permV2C row chunks
for t in range(Z):
	page_addr = msgPass_buffer_ctrl_inst.page_addr_gen(t=t, shift_factor=S_i_j)
	word_addr = msgPass_buffer_ctrl_inst.word_addr_gen(t=t, shift_factor=S_i_j)
	msgPass_buffer[page_addr][word_addr] = t

compV2C_base_addr = 0 # tentative value for ease of simulation
permV2C_base_addr = N_rc+1 # tentative value for ease of simulation
page_raddr = 0 + compV2C_base_addr

print(f"msgPass_buffer before cyclic shift = \n{msgPass_buffer}")
print("------------------------------------------------------------------------")
for u in range(U):
	for i in range(W_s):
		abs_rowChunk_id = u*W_s + i # (stride unit ID, relative row chunk ID) |-> absolute row chunk ID
		S_bs = cyclic_shiftFactor_vec[0][0][abs_rowChunk_id]
		#print(f"Stirde unit ID: {u}, Row chunk ID: {i} |-> Absolute row chunk ID: {abs_rowChunk_id}, S_bs: {S_bs}")
		page_waddr = permV2C_base_addr
		for j in range(P_r):
			y_fetch_compV2C = msgPass_buffer[page_raddr][j]
			hat_j = int((j + S_bs) % P_r)
			msgPass_buffer[page_waddr][hat_j] = y_fetch_compV2C
		page_raddr += 1

print(f"msgPass_buffer after cyclic shift = \n{msgPass_buffer}")

msgPass_buffer before cyclic shift = 
[[ 0  5 10]
 [ 1  6 11]
 [ 2  7 12]
 [ 3  8 13]
 [ 4  9 14]
 [ 0  0  0]
 [ 0  0  0]
 [ 0  0  0]
 [ 0  0  0]
 [ 0  0  0]
 [ 0  0  0]]
------------------------------------------------------------------------
msgPass_buffer after cyclic shift = 
[[ 0  5 10]
 [ 1  6 11]
 [ 2  7 12]
 [ 3  8 13]
 [ 4  9 14]
 [ 0  0  0]
 [ 9 14  4]
 [ 0  0  0]
 [ 0  0  0]
 [ 0  0  0]
 [ 0  0  0]]
